# Testing Vannmiljø API

Outline code for getting data via the Vannmiljø API. Needs improving and integrating into `nivapy`.

In [1]:
import json

import pandas as pd
import requests


def get_data_from_vannmiljo(endpoint, params=None):
    """Vannmiljø endpoints are documented here:

        https://vannmiljowebapi.miljodirektoratet.no/swagger/ui/index#!

    This function can be used with any of the GET endpoints.

    Args
        endpoint: Str. Any of the GET endpoints listed for the API (e.g.
                  'GetMediumList')
        params:   Dict. Valid parameters for the chosen endpoint - see the
                  Swagger UI for details. E.g. {'filter': 'BB'} for the
                  'GetMediumList' endpoint

    Returns
        Dataframe.
    """
    headers = {
        "Content-Type": "application/json",
    }
    url = f"https://vannmiljowebapi.miljodirektoratet.no/api/Public/{endpoint}"
    response = requests.get(url, headers=headers, params=params)

    return pd.DataFrame(response.json())


def get_vannmiljo_api_key():
    """Get API key for POSTing to Vannmiljø."""
    fpath = f"/home/jovyan/shared/common/01_datasets/tokens/vannmiljo_api_token.json"
    with open(fpath) as f:
        data = json.loads(f.read())

    return data["VANNMILJO_API_KEY"]


def post_data_to_vannmiljo(endpoint, data=None):
    """Vannmiljø endpoints are documented here:

        https://vannmiljowebapi.miljodirektoratet.no/swagger/ui/index#!

    This function can be used with any of the POST endpoints.

    Args
        endpoint: Str. Any of the POST endpoints listed for the API (e.g.
                  'GetRegistrations')
        data:     Dict. Valid parameters for the chosen endpoint - see the
                  Swagger UI for details. E.g.
                      {'WaterLocationCodeFilter': ['002-58798']}
                  for the 'GetRegistrations' endpoint

    Returns
        Dataframe.
    """
    headers = {
        "vannmiljoWebAPIKey": get_vannmiljo_api_key(),
        "Content-Type": "application/json",
    }
    url = f"https://vannmiljowebapi.miljodirektoratet.no/api/Public/{endpoint}"
    response = requests.post(url, headers=headers, data=json.dumps(data))

    return pd.DataFrame(response.json()["Result"])

In [2]:
# Example request for any 'GET' endpoint
par_df = get_data_from_vannmiljo("GetParameterList")
par_df.head()

,ParameterID,Name,Description,SortOrder,QualityElementID,SubGroupID,EUcode,CASnr,ComponentID,ParameterID_VannNett,ParameterID_WISE,ParameterLabel_WISE,ParameterID_WISE2,ParameterLabel_WISE2
0,ES100,Hurlberts diversitetsindeks (ES100) marin bløt...,Forventet antall arter en vil finne i en prøve...,None,BVD,None,QE1-3-Diversity (ES100) marine soft-bottom fauna,None,109.0,QE1-3-Diversity (ES100) marine soft-bottom fauna,None,None,None,None
1,FIANTLLE,Antall individer fisk (art) innen en gitt leng...,Antall individer fisk av en bestemt art innenf...,None,FIS,None,Number of fish (spp) for a given size group,None,NaN,None,None,None,None,None
2,FIANTLVE,Antall individer fisk (art) innen en gitt vekt...,Antall individer fisk av en bestemt art innenf...,None,FIS,None,Number of fish (spp) for a given weight class,None,NaN,None,None,None,None,None
3,FIGYTE,Antall gytegroper,Antall gytegroper,None,FIS,None,Number of spawning beds,None,NaN,None,None,None,None,None
4,FIKOND,Kondisjonsfaktor for fisk (art),Lengde-vektforhold av en bestemt art beregnet ...,None,FIS,None,Condition factor fish (spp),None,NaN,None,None,None,None,None


In [3]:
# Query stations
data = {
    "MaxReturnCount": 10,
    # "BoundingBox": {
    #     "xmin": 0,
    #     "ymin": 0,
    #     "xmax": 0,
    #     "ymax": 0,
    #     "zmin": 0,
    #     "zmax": 0,
    #     "mmin": 0,
    #     "mmax": 0,
    # },
    # "WaterLocationIDFilter": [0],
    "WaterLocationCodeFilter": ["002-58798"],
    # "WaterLocationNameFilter": ["string"],
    # "WaterLocationNameAlternativeFilter": ["string"],
    # "WaterLocationCategoryFilter": ["string"],
    # "WaterLocationWaterTypeFilter": ["string"],
    # "FylkeIDFilter": ["string"],
    # "ActivityIDFilter": ["string"],
    # "KommuneIDFilter": ["string"],
    # "VannregionIDFilter": ["string"],
    # "VannomradeIDFilter": ["string"],
    # "VassdragsomradeIDFilter": ["string"],
    # "DescriptionFilter": ["string"],
    # "FromElevation": 0,
    # "ToElevation": 0,
    # "HasWaterRegistrations": True,
    # "HasArchiveElements": True,
    # "FromCreatedDate": "2023-10-19T13:06:43.530Z",
    # "ToCreatedDate": "2023-10-19T13:06:43.530Z",
    # "FromLastModifiedDate": "2023-10-19T13:06:43.530Z",
    # "ToLastModifiedDate": "2023-10-19T13:06:43.530Z",
    # "FromLastDataimportDate": "2023-10-19T13:06:43.530Z",
    # "ToLastDataimportDate": "2023-10-19T13:06:43.530Z",
    # "CreatedByUser": "string",
    # "WaterLocationSourceIDFilter": ["string"],
    # "FeatureType": "string",
    # "Classify": True,
}
stn_df = post_data_to_vannmiljo("GetWaterLocations", data=data)
stn_df

,WaterLocationID,WaterLocationCode,Name,ActivityID,Betegnelse,Description,Classify,CreateDate,LastModifiedDate,Elevation,...,Kommune,VassdragsomradeID,Vassdragsomrade,VannomradeID,Vannomrade,VannregionID,Vannregion,CoordY,CoordX,LastDataimportDate
0,58798,002-58798,"Nybubekken, nedstrøms doserer",KALL,None,,True,2012-12-12T10:40:00,2019-11-04T14:44:00,None,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,6765233.047,292044.8673,2023-05-03T10:39:29


In [4]:
# Query results
data = {
    # "MaxReturnCount": 0,
    # "RegType": 0,
    # "MediumID": "string",
    # "FromDateSamplingTime": "2008-01-01T13:06:43.508Z",
    # "ToDateSamplingTime": "2008-12-31T13:06:43.509Z",
    # "FromRegDate": "2023-10-19T13:06:43.509Z",
    # "ToRegDate": "2023-10-19T13:06:43.509Z",
    # "LatinskNavnID": 0,
    # "ActivityID": "KALK",
    # "SamplingMethodID": "string",
    # "AnalysisMethodID": "string",
    # "RegValueOperator": "string",
    # "RegValue": 0,
    # "RegValue2": 0,
    # "UpperDepthOperator": "string",
    # "UpperDepth": 0,
    # "UpperDepth2": 0,
    # "UpperDepthIncludeNull": True,
    # "LowerDepthOperator": "string",
    # "LowerDepth": 0,
    # "LowerDepth2": 0,
    # "LowerDepthIncludeNull": True,
    # "Employer": "string",
    # "Contractor": "string",
    # "BoundingBox": {
    #     "xmin": 0,
    #     "ymin": 0,
    #     "xmax": 0,
    #     "ymax": 0,
    #     "zmin": 0,
    #     "zmax": 0,
    #     "mmin": 0,
    #     "mmax": 0,
    # },
    # "WaterLocationIDFilter": [0],
    # "ParameterIDFilter": ["string"],
    "WaterLocationCodeFilter": ["002-58798"],
    # "WaterLocationNameAlternativeFilter": ["string"],
    # "WaterLocationCategoryFilter": ["string"],
    # "WaterLocationWaterTypeFilter": ["string"],
    # "FylkeIDFilter": ["string"],
    # "KommuneIDFilter": ["string"],
    # "VannregionIDFilter": ["string"],
    # "VannomradeIDFilter": ["string"],
    # "VassdragsomradeIDFilter": ["string"],
}
wc_df = post_data_to_vannmiljo("GetRegistrations", data=data)
wc_df

,WaterRegistrationID,WaterLocationID,WaterLocationCode,Name,Name_Alternative,SourceID,ActivityID,ActivityName,Employer,Contractor,...,Kommune,VassdragsomradeID,Vassdragsomrade,VannomradeID,Vannomrade,VannregionID,Vannregion,WaterType,CoordX,CoordY
0,47032627,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
1,47031799,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
2,47045906,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
3,47027541,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
4,47044239,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1101,47025063,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
1102,47024821,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
1103,102181904,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047
1104,102178554,58798,002-58798,"Nybubekken, nedstrøms doserer",None,None,KALK,Tiltaksovervåking i kalkede laksevassdrag,Miljødirektoratet,VestfoldLAB AS,...,Hamar,002,Glommavassdraget/Hvaler og Singlefjorden,5107-12,Mjøsa,5107,Innlandet og Viken,R206,292044.8673,6765233.047


In [5]:
sorted(wc_df.columns)

['ActivityID',
 'ActivityName',
 'AnalysisMethodID',
 'AnalysisMethodName',
 'Archive',
 'CASnr',
 'Comment',
 'Contractor',
 'CoordX',
 'CoordY',
 'DetectionLimit',
 'Employer',
 'FeatureType',
 'Fylke',
 'FylkeID',
 'Kommune',
 'KommuneID',
 'LastEditDate',
 'LatinskNavnID',
 'ListName',
 'LowerDepth',
 'MediumID',
 'MediumName',
 'Name',
 'Name_Alternative',
 'NumberOfValues',
 'Origin',
 'ParameterID',
 'ParameterName',
 'QualityElementID',
 'QualityElementTypeID',
 'QuantificationLimit',
 'RegDate',
 'RegValue',
 'SampleID',
 'SamplingMethodID',
 'SamplingMethodName',
 'SamplingTime',
 'SourceID',
 'Unit',
 'UnitID',
 'UpperDepth',
 'Username',
 'ValueOperator',
 'Vannomrade',
 'VannomradeID',
 'Vannregion',
 'VannregionID',
 'Vassdragsomrade',
 'VassdragsomradeID',
 'VitenskapligNavn',
 'WaterCategory',
 'WaterLocationCode',
 'WaterLocationID',
 'WaterRegistrationID',
 'WaterType']

In [6]:
# Just a subset of columns
cols = [
    "WaterLocationID",
    "WaterLocationCode",
    "Name",
    "ActivityID",
    "ActivityName",
    "MediumID",
    "MediumName",
    "ParameterID",
    "ParameterName",
    "SamplingTime",
    "ValueOperator",
    "RegValue",
    "Unit",
]
wc_df[cols].sort_values("SamplingTime")

,WaterLocationID,WaterLocationCode,Name,ActivityID,ActivityName,MediumID,MediumName,ParameterID,ParameterName,SamplingTime,ValueOperator,RegValue,Unit
565,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,N-SNOX,Nitrat + nitritt,2018-01-08T00:00:00,=,27.000,µg/l N
18,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,ALK,Total alkalitet,2018-01-08T00:00:00,=,0.084,mmol/l
279,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,ILAL,Ikke-labilt aluminium,2018-01-08T00:00:00,=,88.000,µg/l Al
341,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,K,Kalium,2018-01-08T00:00:00,=,0.060,mg/l
377,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,KOND,Konduktivitet,2018-01-08T00:00:00,=,1.300,mS/m
...,...,...,...,...,...,...,...,...,...,...,...,...,...
397,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,KOND,Konduktivitet,2022-02-28T00:00:00,=,1.680,mS/m
677,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,PH,pH,2022-02-28T00:00:00,=,6.100,<ubenevnt>
189,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,CA,Kalsium,2022-03-14T00:00:00,=,2.600,mg/l
676,58798,002-58798,"Nybubekken, nedstrøms doserer",KALK,Tiltaksovervåking i kalkede laksevassdrag,VF,Ferskvann,PH,pH,2022-03-14T00:00:00,=,6.300,<ubenevnt>
